In [ ]:
!pip install git+https://github.com/MassimoStel/emoatlas

In [ ]:
!python -m spacy download it_core_news_lg


In [ ]:
!python -m spacy download en_core_web_lg

In [3]:
import nltk
# nltk.download('wordnet') # for English
nltk.download('omw-1.4') #This is for EmoAtlas in Italian

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
import matplotlib
from emoatlas import EmoScores
import emoatlas

In [5]:
emos = EmoScores(language='italian')

In [ ]:
emos.emotions("Porcheria elettoralistica. Ottimo in caso di necessità fisiologiche")
# emos.zscores("Porcheria elettoralistica. Ottimo in caso di necessità fisiologiche")

{'anger': 0,
 'trust': 0,
 'surprise': 0,
 'disgust': 0,
 'joy': 0,
 'sadness': 1,
 'fear': 0,
 'anticipation': 0}

In [6]:
import pandas as pd


In [22]:
comments = pd.read_csv("product_reviews_comments_topic_FINAL.csv", delimiter='|')

print(comments)

subset = comments.iloc[:10, [2, 3]]

print(subset)

      index   book_code                            title_review  \
0         0  B0CLML6FM8                       Utile e ben fatto   
1         1  B0CLML6FM8                                 attuale   
2         2  B0CLML6FM8                   israele e palestinesi   
3         3  B0CLML6FM8                        Molto articolato   
4         4  B0CLML6FM8  Buono per una visione a volo d'uccello   
...     ...         ...                                     ...   
3743   3743  8807036525                              Leggetelo.   
3744   3744  8807036525                                      Ni   
3745   3745  8807036525                    Un'occasione mancata   
3746   3746  8807036525           Fantastico romanzo d'esordio!   
3747   3747  8807036525              libro "La strada giovane".   

                                     description_review  \
0     E' uno dei resoconti piu' ordinati che abbia m...   
1     Bel libro attuale e snocciola le problematiche...   
2     interessante

In [23]:
comments[1:2]

,index,book_code,title_review,description_review,useful_description,useful_num,idx_2,topic_num,topic_description
1,1,B0CLML6FM8,attuale,Bel libro attuale e snocciola le problematiche...,Una persona l'ha trovato utile,1,2,5,Conflitto Israelo-Palestinese


In [15]:
import json

In [24]:
from nltk.tokenize import word_tokenize
from string import punctuation

df_comments_emoscores = []
for idx, rows in comments.iterrows():
  title_review = rows[2]
  description_review = ' '

  if pd.isna(rows[3]) == False:
    description_review = rows[3]

  # print(title_review + ' ' + description_review)
  # print(emos.emotions(title_review + ' ' + description_review))
  # print(emos.zscores(title_review + ' ' + description_review))

  complete_text_clean = (title_review + ' ' + description_review).replace('.', ' ')

  tenant_res_scores = emoscores_and_tokenlen(complete_text_clean)
  rows["EMO_ZSCORES"] = tenant_res_scores[0] # emozscores
  rows["MEANINGFUL_EMOTIONS"] = tenant_res_scores[1] # meaningful_emo
  rows["TOKEN_LEN"] = tenant_res_scores[2] # len(filtered_tokens)
  rows["COMPLEX_EMOTIONS"] = tenant_res_scores[3] # complex emotions
  rows["NODE_INDEX"] = rows[1] + 'RW' + str(idx)

  print(rows)

  with open("product_reviews_comments_topic_emozscores.jsonl", "a") as file:
    file.write(json.dumps(rows.to_dict()) + '\n')


Output streaming troncato alle ultime 5000 righe.
MEANINGFUL_EMOTIONS                                                   []
TOKEN_LEN                                                             25
COMPLEX_EMOTIONS                                                      []
NODE_INDEX                                              8804751169RW3414
Name: 3414, dtype: object
index                                                               3415
book_code                                                     8804751169
title_review                                                Sempre utile
description_review     La scrittura di Cazzullo è sempre schietta, di...
useful_description                        Una persona l'ha trovato utile
useful_num                                                             1
idx_2                                                               3416
topic_num                                                              4
topic_description                              G

In [17]:
from nltk.tokenize import word_tokenize
from string import punctuation
import json

nltk.download('punkt_tab')

def emoscores_and_tokenlen(text):
  complete_text_clean = (text).replace('.', ' ').replace(',', ' ').replace(';', ' ')
  emozscores = emos.zscores(complete_text_clean)

  # print(emozscores)
  meaningful_emo = []
  for key, value in emozscores.items():
    #  print(key)
    #  print(value)
    if (round(value, 2) > 1.96 ):
      # print(f'Emozione significativa {key} - {str(round(value,2))}')
      meaningful_emo.append(key+'_above')
    elif (round(value, 2) < -1.96):
      meaningful_emo.append(key+'_under')

  complex_emotions = find_complex_emotions(meaningful_emo)


  # print(meaningful_emo)

  tokens_it = word_tokenize(complete_text_clean, language='italian')

  # Filter out punctuation
  filtered_tokens = [token for token in tokens_it if token not in punctuation]

  # print(filtered_tokens)
  # print(len(filtered_tokens))

  return [emozscores, meaningful_emo,len(filtered_tokens), complex_emotions]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [11]:
from typing import List

def find_complex_emotions(emotions: List[str]) -> List[str]:
    complex_emotions = []

    # Define the mappings for complex emotions
    emotion_combinations = {
        'optimism': (('anticipation_above', 'joy_above'),),
        'disapproval': (('surprise_above', 'sadness_above'),),
        'hope': (('anticipation_above', 'trust_above'),),
        'unbelief': (('surprise_above', 'disgust_above'),),
        'anxiety': (('anticipation_above', 'fear_above'),),
        'hate': (('surprise_above', 'anger_above'),),
        'friendliness': (('joy_above', 'trust_above'),),
        'remorse': (('sadness_above', 'disgust_above'),),
        'guilt': (('joy_above', 'fear_above'),),
        'excitement': (('joy_above', 'fear_above'),),
        'envy': (('sadness_above', 'anger_above'),),
        'delight': (('joy_above', 'surprise_above'),),
        'pessimism': (('sadness_above', 'anticipation_above'),),
        'submission': (('trust_above', 'fear_above'),),
        'contempt': (('disgust_above', 'anger_above'),),
        'curiosity': (('trust_above', 'surprise_above'),),
        'cynicism': (('disgust_above', 'anticipation_above'),),
        'resignation': (('trust_above', 'sadness_above'),),
        'morbidness': (('disgust_above', 'joy_above'),),
        'awe': (('fear_above', 'surprise_above'),),
        'despair': (('fear_above', 'sadness_above'),),
        'aggressiveness': (('anger_above', 'anticipation_above'),),
        'pride': (('anger_above', 'joy_above'),),
        'shame': (('fear_above', 'disgust_above'),),
        'dominance': (('anger_above', 'trust_above'),),
        'bittersweetness': (('joy_above', 'sadness_above'),),
        'ambivalence': (('trust_above', 'disgust_above'),),
        'frozenness': (('fear_above', 'anger_above'),),
        'confusion': (('surprise_above', 'anticipation_above'),),
    }

    # Check for each complex emotion
    for emotion, combinations in emotion_combinations.items():
        for combo in combinations:
            if all(e in emotions for e in combo):
                complex_emotions.append(emotion)
                break

    return complex_emotions

# Example usage:
emotions = ["surprise_above", "disgust_above"]
complex_emotions = find_complex_emotions(emotions)
print(complex_emotions)


['unbelief']


In [ ]:

text = "Il libro è una chiara espressione della libertà di pensiero di una persona. Che ha trovato il coraggio di scrivere ciò che per lui è o dovrebbe essere la normalità.Ho letto il libro nella sua interezza e in alcuni capitoli il mio pensiero combacia o si avvicina molto a quello dello scrittore, in altri invece mi trovo a essere contrario.Ma la cosa importante è che, per me, un libro deve lasciarti qualcosa. Questo libro lascia solo una sensazione di rabbia verso la società e verso le scelte dei governi europei. Non insegna nulla, non cerca soluzioni alle ""problematiche"" raccontate, le spiega e basta, spesso con esempi di una banalità unica.Se volete sapere quello che pensa lo scrittore compratelo, ma vi consiglio di risparmiare queste 20€ e andare a fare quattro chiacchiere al bar, le tematiche affrontate saranno le stesse e le 20€ potete spenderle in birra."
# emos.draw_formamentis_flower(text)
print(emos.emotions(text))
print(emos.zscores(text))

emozscores = emos.zscores(text)
print(emozscores)
meaningful_emo = []
for key, value in emozscores.items():
  print(key)
  print(value)
  if (round(value, 2) > 1.96 ):
    print(f'Emozione significativa {key} - {str(round(value,2))}')
    meaningful_emo.append(key+'_above')
  elif (round(value, 2) < -1.96):
    meaningful_emo.append(key+'_under')


print(meaningful_emo)
#emos.draw_statistically_significant_emotions(text)

{'anger': 3, 'trust': 9, 'surprise': 4, 'disgust': 1, 'joy': 7, 'sadness': 3, 'fear': 2, 'anticipation': 6}
{'anger': np.float64(-1.2280321787076154), 'trust': np.float64(1.8568524828991504), 'surprise': np.float64(1.1430742203929574), 'disgust': np.float64(-1.6340264787165353), 'joy': np.float64(1.3628872918502333), 'sadness': np.float64(-0.9901090392539246), 'fear': np.float64(-2.27086028480055), 'anticipation': np.float64(1.6262437823524365)}
{'anger': np.float64(-1.2280321787076154), 'trust': np.float64(1.8568524828991504), 'surprise': np.float64(1.1430742203929574), 'disgust': np.float64(-1.6340264787165353), 'joy': np.float64(1.3628872918502333), 'sadness': np.float64(-0.9901090392539246), 'fear': np.float64(-2.27086028480055), 'anticipation': np.float64(1.6262437823524365)}
anger
-1.2280321787076154
trust
1.8568524828991504
surprise
1.1430742203929574
disgust
-1.6340264787165353
joy
1.3628872918502333
sadness
-0.9901090392539246
fear
-2.27086028480055
anticipation
1.626243782352

In [ ]:
from nltk.tokenize import word_tokenize
from string import punctuation

for punct in punctuation:
  print(punct)
# Sample text in Italian
text_it = "Ciao,mondo! Questo è un esempio.di frase in italiano.".replace('.', ' ').replace(',', ' ')

tokens_it = word_tokenize(text_it, language='italian')

# Filter out punctuation
filtered_tokens = [token for token in tokens_it if token not in punctuation]

print(filtered_tokens)
len(filtered_tokens)

!
"
#
$
%
&
'
(
)
*
+
,
-
.
/
:
;
<
=
>
?
@
[
\
]
^
_
`
{
|
}
~
['Ciao', 'mondo', 'Questo', 'è', 'un', 'esempio', 'di', 'frase', 'in', 'italiano']


10